In [1]:
import numpy as np
import gensim 
from gensim.models.word2vec import Word2Vec 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Activation, Dense

import time

import helpers as HL
import cleaning as CL

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [ ]:
filenames = ['train_pos.txt', 'train_neg.txt'] 

corpus, file_lengths = HL.create_corpus(filenames)

In [ ]:
cluster_file_path = '50mpaths2.txt'
cluster_dict = CL.create_dictionary(cluster_file_path)

#clusterised_corpus = CL.create_clusterized_corpus(corpus, cluster_dict)

#clusterised_corpus[0]

In [ ]:
class IterableCorpus():
    
    def __init__(self, corpus):
        self.corpus = corpus
        
    def __iter__(self):
        for tweet in self.corpus:
            tweet_words = tweet.split()
            yield [word.decode('utf-8') for word in tweet_words]

In [ ]:
iterable_corpus = IterableCorpus(clusterised_corpus)

# SMÆKK INN HYPERPARAMETERE HER!!! 
w2v_model = Word2Vec(iterable_corpus, size=300, window=5, min_count=10)

In [ ]:
#Training-set numbers
nr_of_positive_training_examples = 100000
nr_of_training_examples = 200000

#Making labels
labels = np.zeros(nr_of_training_examples);
labels[0:nr_of_positive_training_examples]=1;
labels[nr_of_positive_training_examples:nr_of_training_examples]=0; 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(np.array(clusterised_corpus),
                                                    np.array(labels), test_size=0.2)


#### So we have our word2vec model, giving us a representation for each word in the whole corpus. We now need to create a representation of each Tweet in the dataset. 

We will do this with the following steps: 
- Creating a TFIDF model for each word
- Combining the word2vec-vectors for each word, weighting them by their TFIDF score
- Run that shit through a neural network fuck yeaaaaah

In [3]:
#Creating a vectorizer to use for Tf-IDF analysis on words
vectorizer = TfidfVectorizer(
        min_df = 10, # removing word that occure less then 10 times 
        max_df = 1.5, # remove words that are too frequent ( more then 1.5 * number of tweets )
        sublinear_tf=True, # scale the term frequency in logarithmic scale
        use_idf =True
        #stop_words = custom_stop_words # Removing stop-words
    )

In [4]:
corpus_tf_idf = vectorizer.fit_transform(clusterised_corpus)

NameError: name 'clusterised_corpus' is not defined

In [ ]:
corpus_tf_idf

In [ ]:
tfidf_dict = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

print('Size: ', len(tfidf_dict))

#### Now we want to create a vector for each tweet, by combining the word vecs weighted by the tfidf-dict 

In [ ]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens.split():
        try:       
            word = word.decode('utf-8')
            word_vec = w2v_model[word].reshape((1, size))             
            idf_weighted_vec = word_vec * tfidf_dict[word]
            vec += idf_weighted_vec
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [ ]:
# NEEDS TO BE THE SAME AS WHEN CREATING WORD2VEC
n_dim = 300

total_score = 0

train_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in x_train])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in x_test])
test_vecs_w2v = scale(test_vecs_w2v)


model = Sequential()
model.add(Dense(32, activation='relu', input_dim=n_dim))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
      loss='binary_crossentropy',
      metrics=['accuracy'])

model.fit(train_vecs_w2v, y_train, epochs=15, batch_size=32, verbose=1)

score = model.evaluate(test_vecs_w2v, y_test, batch_size=128, verbose=1)

print(score)

### FOR KEGGLE DELIVERY

In [4]:
training_filenames = ['train_pos_full.txt', 'train_neg_full.txt'] 
training_corpus, training_file_lengths = HL.create_corpus(training_filenames)

test_corpus, test_corpus_length = HL.create_corpus(['test_data.txt'])

In [5]:
cluster_file_path = '50mpaths2.txt'
cluster_dict = CL.create_dictionary(cluster_file_path)

In [6]:
clusterised_training_corpus = CL.create_clusterized_corpus(training_corpus, cluster_dict)
clusterised_test_corpus = CL.create_clusterized_corpus(test_corpus, cluster_dict)

len(clusterised_training_corpus)

2500000

In [7]:
class IterableCorpus():
    
    def __init__(self, corpus):
        self.corpus = corpus
        
    def __iter__(self):
        for tweet in self.corpus:
            tweet_words = tweet.split()
            yield [word.decode('utf-8') for word in tweet_words]

In [8]:
iterable_training_corpus = IterableCorpus(clusterised_training_corpus)
iterable_test_corpus = IterableCorpus(clusterised_test_corpus)

In [9]:
w2v_test_model = Word2Vec(iterable_test_corpus, size=300, window=5, min_count=10)

In [10]:
w2v_training_model = Word2Vec(iterable_training_corpus, size=300, window=5, min_count=10)

In [11]:
#Training-set numbers
nr_of_positive_training_examples = 1250000
nr_of_positive_training_examples = 1250000
nr_of_training_examples = 2500000
nr_of_test_examples = 10000

#Making labels
labels = np.zeros(nr_of_training_examples);
labels[0:nr_of_positive_training_examples]=1;
labels[nr_of_positive_training_examples:nr_of_training_examples]=0; 

In [12]:
training_vectorizer = TfidfVectorizer(
        min_df = 10, # removing word that occure less then 10 times 
        max_df = 1.5, # remove words that are too frequent ( more then 1.5 * number of tweets )
        sublinear_tf=True, # scale the term frequency in logarithmic scale
        use_idf =True
        #stop_words = custom_stop_words # Removing stop-words
    )

In [13]:
test_vectorizer = TfidfVectorizer(
        min_df = 10, # removing word that occure less then 10 times 
        max_df = 1.5, # remove words that are too frequent ( more then 1.5 * number of tweets )
        sublinear_tf=True, # scale the term frequency in logarithmic scale
        use_idf =True
        #stop_words = custom_stop_words # Removing stop-words
    )

In [14]:
corpus_training_tf_idf = training_vectorizer.fit_transform(clusterised_training_corpus)
corpus_test_tf_idf = test_vectorizer.fit_transform(clusterised_test_corpus)

In [15]:
tfidf_training_dict = dict(zip(training_vectorizer.get_feature_names(), training_vectorizer.idf_))
tfidf_test_dict = dict(zip(test_vectorizer.get_feature_names(), test_vectorizer.idf_))

In [16]:
def buildWordVectorKeggle(tokens, size, model, tfidfdict):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens.split():
        try:       
            word = word.decode('utf-8')
            word_vec = model[word].reshape((1, size))             
            idf_weighted_vec = word_vec * tfidfdict[word]
            vec += idf_weighted_vec
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [17]:
# NEEDS TO BE THE SAME AS WHEN CREATING WORD2VEC
n_dim = 300

total_score = 0

train_vecs_w2v = np.concatenate([buildWordVectorKeggle(z, n_dim, w2v_training_model, tfidf_training_dict) for z in clusterised_training_corpus])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVectorKeggle(z, n_dim, w2v_test_model, tfidf_test_dict) for z in clusterised_test_corpus])
test_vecs_w2v = scale(test_vecs_w2v)

#score = model.evaluate(test_vecs_w2v, y_test, batch_size=128, verbose=1)

#print(score)

In [24]:
train_vecs_w2v.shape

(2500000, 300)

In [27]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=n_dim))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
      loss='binary_crossentropy',
      metrics=['accuracy'])

model.fit(train_vecs_w2v, labels, epochs=6, batch_size=32, verbose=1)

Epoch 1/6
2500000/2500000 [==============================] - 638s 255us/step - loss: 0.4132 - acc: 0.8040
Epoch 2/6
2500000/2500000 [==============================] - 220s 88us/step - loss: 0.3985 - acc: 0.8136
Epoch 3/6
2500000/2500000 [==============================] - 181s 73us/step - loss: 0.3947 - acc: 0.81590s - loss: 0.3946 - acc: 0.81
Epoch 4/6
2500000/2500000 [==============================] - 158s 63us/step - loss: 0.3921 - acc: 0.8171
Epoch 5/6
2500000/2500000 [==============================] - 164s 66us/step - loss: 0.3906 - acc: 0.8182
Epoch 6/6
2500000/2500000 [==============================] - 164s 66us/step - loss: 0.3894 - acc: 0.8188


In [47]:
predictions = model.predict(test_vecs_w2v)

print(predictions)

rounded = [-1 if round(x[0])==0 else 1 for x in predictions]

print(rounded)

[[ 0.51762182]
 [ 0.47987047]
 [ 0.67128795]
 ..., 
 [ 0.67458504]
 [ 0.68462485]
 [ 0.46197894]]
[1, -1, 1, 1, -1, -1, -1, 1, -1, -1, 1, 1, -1, 1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, -1, -1, 1, -1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, -1, 1, 1, -1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, -1, -1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, -1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1

In [49]:
ids = list(range(1,nr_of_test_examples+1))
y_pred = rounded
name = "keggle_submission_neural.csv"

HL.create_csv_submission(ids, y_pred, name)